## LSTM

In [57]:
# Restore variable
%store -r _X
%store -r _Y

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import utils as u
from numpy import mean
from numpy import std
import importlib
importlib.reload(u)

<module 'utils' from 'C:\\Users\\ghait\\Desktop\\MSc\\CS5500\\implementation\\Final\\utils.py'>

## Normalisation

In [28]:
X_min = _X.min(axis=(1, 2), keepdims=True)
X_max = _X.max(axis=(1, 2), keepdims=True)
X_norm = (_X - X_min)/(X_max-X_min)

## Standarisation

In [58]:
X_stand = []
for i in range (len(_X)):
    numerator = _X[i,:,:] - np.mean(_X[i,:,:], axis=0)
    denominator = np.std(_X[i,:,:], axis=0)
    temp = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0) # avoid division on zero
    X_stand.append(temp)
X_stand = np.array(X_stand)
X_stand = X_stand[:,:,:] # because the results were in 3D array

# Spliting the data Cross Subject

In [59]:
"""
Any of these tensors can be an input for the function below 
X_norm -- normalised 
X_stand -- standardised 
_X -- original data without any manipulation 
"""
import importlib
importlib.reload(u)
# this function split the dataset to training and test set
X_trn,Y_trn,X_tst,Y_tst = u.train_test(_X, _Y) 
print(X_trn.shape)
print(X_tst.shape)

(1800, 179, 66)
(200, 179, 66)


In [60]:
X_test = X_tst
X_train = X_trn
Y_train = u.convert_to_one_hot(Y_trn[:,2],2).T
Y_test = u.convert_to_one_hot(Y_tst[:,2], 2).T

## Batch shape (taining and test)

In [61]:
print ("Number of training examples = " + str(X_train.shape[0]))
print ("Number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

Number of training examples = 1800
Number of test examples = 200
X_train shape: (1800, 179, 66)
Y_train shape: (1800, 2)
X_test shape: (200, 179, 66)
Y_test shape: (200, 2)


# Model Design

In [33]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras import utils as np_utils
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [63]:
test_scores = [] 
train_scores = [] 

# Hyperparameter 
verbose = 0 
epochs = 200 
batch_size = 50
repeats = 3

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs =  Y_train.shape[1]

for r in range(repeats):
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(X_train, Y_train, epochs, batch_size, verbose=verbose)
    
    # evaluate model training set
    _, train_accuracy = model.evaluate(X_train, Y_train, batch_size=batch_size, verbose=0)
    t_score = train_accuracy * 100.0
    print('># Train Accuracy iteration %d: %.3f' % (r+1, t_score))
    train_scores.append(t_score)
    
    # evaluate model test set
    _, test_accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    score = test_accuracy * 100.0
    print('># Test Accuracy iteration  %d: %.3f' % (r+1, score))
    test_scores.append(score)

# summarize train results
print(train_scores)
m, s = mean(train_scores), std(train_scores)
print('Train Accuracy: %.2f%% (+/-%.2f)' % (m, s))

# summarize test results
print(test_scores)
m, s = mean(test_scores), std(test_scores)
print('Test Accuracy: %.2f%% (+/-%.2f)' % (m, s))

># Train Accuracy iteration 1: 50.000
># Test Accuracy iteration  1: 50.000
># Train Accuracy iteration 2: 50.722
># Test Accuracy iteration  2: 49.000


KeyboardInterrupt: 